# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 

The CNN-RNN architecture implemented for this project is similar to those presented in the literature  and recommended in this notebook \[1-3\].

**Encoder model (CNN)**: I used the provided pretrained ResNet-50 network as the CNN encoder. ResNet-50 (introduced in [this paper](https://arxiv.org/pdf/1512.03385.pdf)) is a residual deep neural network with 50 layers, including convoltional layers and residual pass through layers. Pretained ResNet models are available on the [PyTorch website](https://pytorch.org/vision/stable/models.html). I added a fully connected layer after the pretrained network that outputs `embed_size` for our image embeddings. Additionally, I added a one-dimensional [batch normalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) layer after the fully connected layer to make the network more robust during training.

**Decoder model (RNN)**: I used a two-layer LSTM network as the RNN encoder network. The function of this network is to input image embeddings and output descriptive text captions. The first part of the network is an [embedding layer](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html), which is a lookup table that stores embeddings of a fixed dictionary and size. The image feature embeddings are concatinated into tensors and fed through the [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) network. I used two-layer stacked LSTM to improve the accuracy of the LSTM network. The LSTM also includes dropout followed by a seperate dropout layer, both with a dropout probability of 0.3. Lastly, the output of the LSTM is fed through a fully connected layer that inputs the `hidden_size` and outputs to `vocab_size`.

**Hyperparameter Values**: I selected a `batch_size` of 64 because it was recommended in \[2\] and helps improve training performance. I chose a minimum word count threshold `vocab_threshold` of 5 because it empirically provides good results and is recommended in \[3\]. The `embed_size` and `hidden_size` values are 512 as recommended in \[1\]. This value provides a good trade-off that allows the network to accurately learn information from the dataset while limiting the number of unknowns in the network. The model is trained for 3 epochs (`num_epochs`) as recommended in this notebook. The training took approximately ~9 hours using the GPU provided in this workspace. The training results in this notebook are cut off in the middle of the second epoch. However, the *training_log.txt* provides a training log for all three epochs and demonstrates a reduction of the loss function and good training results.

References:

\[1\] [Vinyals, Oriol, et al. "Show and Tell: A neural image caption generator." Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. 2015.](https://arxiv.org/pdf/1411.4555.pdf)

\[2\] [Xu, Kelvin, et al. "Show, Attend and Tell: Neural image caption generation with visual attention." International Conference on Machine Learning. PMLR, 2015.](https://arxiv.org/pdf/1502.03044.pdf)

\[3\] [Vinyals, Oriol, et al. "Show and Tell: Lessons learned from the 2015 mscoco image captioning challenge." IEEE Transactions on Pattern Analysis and Machine Intelligence 39.4 (2016): 652-663.](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636)


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I used the default transform provided in this notebook because it provides image scaling, random cropping, random vertical and horizontal flips, and color normalization. This transformation provides good performance and does not require improvement.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** Since the CNN is a pretrained ResNet-50 network, the parameters in this model do not need to be trained. Therefor, I selected the trainable parameters in the encoder network to be the linear layer (*encoder.embed*) and batch normalization (*encoder.bn*) layer parameters. Additionally, all of the decoder parameters are trainable parameters.


### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I used an [Adam optimizer](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam) because it has given me good results training other CNN and RNN networks and it converged faster than [stochastic gradient descent (SGD)](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD) with momentum in my experiments training the network. I used the default parameters including a learning rate (*lr*) of 0.001. Additionally, the Adam optimizer is used in \[2\].

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.bn.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999))

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 411/414113 [00:00<01:40, 4104.46it/s]

  0%|          | 897/414113 [00:00<01:35, 4304.48it/s]

  0%|          | 1391/414113 [00:00<01:32, 4475.40it/s]

  0%|          | 1886/414113 [00:00<01:29, 4607.19it/s]

  1%|          | 2372/414113 [00:00<01:28, 4678.23it/s]

  1%|          | 2858/414113 [00:00<01:26, 4729.98it/s]

  1%|          | 3348/414113 [00:00<01:25, 4777.54it/s]

  1%|          | 3874/414113 [00:00<01:23, 4910.58it/s]

  1%|          | 4370/414113 [00:00<01:23, 4924.58it/s]

  1%|          | 4876/414113 [00:01<01:22, 4963.65it/s]

  1%|▏         | 5384/414113 [00:01<01:21, 4995.79it/s]

  1%|▏         | 5877/414113 [00:01<01:24, 4806.61it/s]

  2%|▏         | 6368/414113 [00:01<01:24, 4836.52it/s]

  2%|▏         | 6869/414113 [00:01<01:23, 4885.30it/s]

  2%|▏         | 7356/414113 [00:01<01:23, 4854.03it/s]

  2%|▏         | 7865/414113 [00:01<01:22, 4920.20it/s]

  2%|▏         | 8381/414113 [00:01<01:21, 4

 31%|███       | 128650/414113 [00:26<00:57, 4991.32it/s]

 31%|███       | 129150/414113 [00:26<00:57, 4965.81it/s]

 31%|███▏      | 129678/414113 [00:26<00:56, 5055.85it/s]

 31%|███▏      | 130185/414113 [00:26<00:56, 5003.32it/s]

 32%|███▏      | 130692/414113 [00:26<00:56, 5019.69it/s]

 32%|███▏      | 131195/414113 [00:26<00:56, 5022.18it/s]

 32%|███▏      | 131698/414113 [00:26<00:57, 4948.68it/s]

 32%|███▏      | 132206/414113 [00:26<00:56, 4984.79it/s]

 32%|███▏      | 132705/414113 [00:26<00:56, 4949.25it/s]

 32%|███▏      | 133201/414113 [00:26<00:57, 4925.82it/s]

 32%|███▏      | 133700/414113 [00:27<00:56, 4943.43it/s]

 32%|███▏      | 134195/414113 [00:27<00:56, 4931.24it/s]

 33%|███▎      | 134720/414113 [00:27<00:55, 5021.17it/s]

 33%|███▎      | 135230/414113 [00:27<00:55, 5042.18it/s]

 33%|███▎      | 135739/414113 [00:27<00:55, 5053.40it/s]

 33%|███▎      | 136254/414113 [00:27<00:54, 5079.42it/s]

 33%|███▎      | 136766/414113 [00:27<00:54, 5091.25it/s

 62%|██████▏   | 256457/414113 [00:51<00:31, 5045.49it/s]

 62%|██████▏   | 256962/414113 [00:51<00:31, 5035.27it/s]

 62%|██████▏   | 257494/414113 [00:52<00:30, 5116.65it/s]

 62%|██████▏   | 258007/414113 [00:52<00:30, 5101.16it/s]

 62%|██████▏   | 258518/414113 [00:52<00:32, 4741.17it/s]

 63%|██████▎   | 259016/414113 [00:52<00:32, 4810.26it/s]

 63%|██████▎   | 259529/414113 [00:52<00:31, 4900.17it/s]

 63%|██████▎   | 260023/414113 [00:52<00:31, 4906.72it/s]

 63%|██████▎   | 260527/414113 [00:52<00:31, 4944.36it/s]

 63%|██████▎   | 261024/414113 [00:52<00:30, 4945.00it/s]

 63%|██████▎   | 261526/414113 [00:52<00:30, 4965.60it/s]

 63%|██████▎   | 262024/414113 [00:53<00:34, 4462.90it/s]

 63%|██████▎   | 262544/414113 [00:53<00:32, 4659.19it/s]

 64%|██████▎   | 263040/414113 [00:53<00:31, 4743.76it/s]

 64%|██████▎   | 263550/414113 [00:53<00:31, 4844.89it/s]

 64%|██████▍   | 264040/414113 [00:53<00:31, 4806.90it/s]

 64%|██████▍   | 264530/414113 [00:53<00:30, 4832.77it/s

 93%|█████████▎| 383308/414113 [01:17<00:06, 4998.73it/s]

 93%|█████████▎| 383813/414113 [01:17<00:06, 5011.73it/s]

 93%|█████████▎| 384315/414113 [01:17<00:06, 4930.64it/s]

 93%|█████████▎| 384809/414113 [01:18<00:05, 4932.12it/s]

 93%|█████████▎| 385306/414113 [01:18<00:05, 4940.80it/s]

 93%|█████████▎| 385826/414113 [01:18<00:05, 5015.76it/s]

 93%|█████████▎| 386340/414113 [01:18<00:05, 5050.56it/s]

 93%|█████████▎| 386846/414113 [01:18<00:05, 5039.43it/s]

 94%|█████████▎| 387353/414113 [01:18<00:05, 5048.13it/s]

 94%|█████████▎| 387859/414113 [01:18<00:05, 5020.11it/s]

 94%|█████████▍| 388362/414113 [01:18<00:05, 5020.79it/s]

 94%|█████████▍| 388879/414113 [01:18<00:04, 5063.14it/s]

 94%|█████████▍| 389408/414113 [01:18<00:04, 5128.13it/s]

 94%|█████████▍| 389922/414113 [01:19<00:04, 5085.63it/s]

 94%|█████████▍| 390434/414113 [01:19<00:04, 5094.60it/s]

 94%|█████████▍| 390944/414113 [01:19<00:04, 5065.19it/s]

 95%|█████████▍| 391454/414113 [01:19<00:04, 5073.39it/s

Done (t=0.84s)
creating index...
index created!
Obtaining caption lengths...


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/6471], Loss: 4.1844, Perplexity: 65.6542
Epoch [1/3], Step [200/6471], Loss: 4.2649, Perplexity: 71.16093
Epoch [1/3], Step [300/6471], Loss: 3.5860, Perplexity: 36.09047
Epoch [1/3], Step [400/6471], Loss: 3.9075, Perplexity: 49.77303
Epoch [1/3], Step [500/6471], Loss: 3.6566, Perplexity: 38.7310
Epoch [1/3], Step [600/6471], Loss: 3.2409, Perplexity: 25.5573
Epoch [1/3], Step [700/6471], Loss: 3.3017, Perplexity: 27.1586
Epoch [1/3], Step [800/6471], Loss: 3.0736, Perplexity: 21.6198
Epoch [1/3], Step [900/6471], Loss: 2.9552, Perplexity: 19.2059
Epoch [1/3], Step [1000/6471], Loss: 2.9027, Perplexity: 18.2232
Epoch [1/3], Step [1100/6471], Loss: 3.2778, Perplexity: 26.5174
Epoch [1/3], Step [1200/6471], Loss: 2.8794, Perplexity: 17.8044
Epoch [1/3], Step [1300/6471], Loss: 2.8414, Perplexity: 17.1394
Epoch [1/3], Step [1400/6471], Loss: 2.5894, Perplexity: 13.3211
Epoch [1/3], Step [1500/6471], Loss: 2.7909, Perplexity: 16.2965
Epoch [1/3], Step [1600/6471], 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.